# **Organización de Datos 75.06 - Trabajo Práctico N°1**
P36. Consiga una matriz cuyas columnas sean los cast id y crew ids (diferenciados para no colisionar), que tenga una fila por cada película y que los valores sean la cantidad de roles para los que esa persona participa en la película. Estime cuánto ocuparía en RAM teniendo en cuenta que cada número pesa 4 bytes, y en caso de no entrar construyala como matriz dispersa.

In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype

from google.colab import drive
drive.mount('/content/drive')

cast = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/TP1/csv/cast.csv")
crew = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/TP1/csv/crew.csv")

Mounted at /content/drive


In [2]:
cast_limpio = cast[ ['id', 'cast_id'] ].dropna()
cast_limpio = cast_limpio.groupby(['cast_id', 'id'], sort = False).size().to_frame().reset_index()
cast_limpio.columns = ['cast_id', 'id', 'roles_quantity']
cast_limpio["cast_id"] = cast_limpio["cast_id"].map(lambda x : "cast_" + str(x))
cast_limpio = cast_limpio.rename(columns = {"cast_id":"person_id"})

crew_limpio = crew[ ['id', 'crew_id'] ].dropna()
crew_limpio = crew_limpio.groupby(['crew_id', 'id'], sort = False).size().to_frame().reset_index()
crew_limpio.columns = ['crew_id', 'id', 'roles_quantity']
crew_limpio["crew_id"] = crew_limpio["crew_id"].map(lambda x : "crew_" + str(x))
crew_limpio = crew_limpio.rename(columns = {"crew_id":"person_id"})

cast_crew = pd.concat([cast_limpio, crew_limpio])

person_id_c = CategoricalDtype(sorted(cast_crew.person_id.unique()), ordered = True)
id_c = CategoricalDtype(sorted(cast_crew.id.unique()), ordered = True)

fil = cast_crew.id.astype(id_c).cat.codes
col = cast_crew.person_id.astype(person_id_c).cat.codes
sparse_matrix = csr_matrix((cast_crew["roles_quantity"], (fil, col)), shape=(id_c.categories.size, person_id_c.categories.size))

resultado = pd.DataFrame.sparse.from_spmatrix(sparse_matrix, index=id_c.categories, columns=person_id_c.categories)

peso = len(resultado) * len(resultado.columns) * 4
print('El peso de la matriz es de', peso, 'bytes, o', peso / pow(10, 9), 'gigabytes.')

resultado

El peso de la matriz es de 66208120252 bytes, o 66.208120252 gigabytes.


,cast_1,cast_10,cast_100,cast_10000,cast_1000007,cast_1000061,cast_1000083,cast_1000089,cast_1000109,cast_1000145,...,crew_999882,crew_999886,crew_999909,crew_99993,crew_99994,crew_999947,crew_999948,crew_99996,crew_999980,crew_999981
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465044,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
467731,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
468343,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
468707,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
